# **Bitcoin price forecasting - Random Forest**
### Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



# Dependencies, Libraries and Tools

In [ ]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
SLOW_OPERATION = False

In [ ]:
#Install some useful dependencies
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

# !pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
# !apt install openjdk-8-jdk-headless -qq
# import os
# os.environ["JAVA_HOME"] = JAVA_HOME

In [ ]:
# Install Spark and related dependencies
!pip install pyspark

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
from pyspark.ml import Pipeline

# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPriceForecasting").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=f6e43b68f52adf4c611a08574b27370ac7709a27283a6317683c13b6666fd08b
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# Link to Google Drive

In [ ]:
# Define GDrive paths
GDRIVE_DIR = "/content/drive"

GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"
GDRIVE_DATASET_TEMP_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/temp"
GDRIVE_DATASET_OUTPUT_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/output"

GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_1h"
GDRIVE_DATASET_NAME_TRAIN = GDRIVE_DATASET_NAME + "_train"
GDRIVE_DATASET_NAME_VALID = GDRIVE_DATASET_NAME + "_valid"

GDRIVE_DATASET_NAME_EXT = "/" + GDRIVE_DATASET_NAME + ".parquet"
GDRIVE_DATASET_NAME_EXT_TRAIN  = "/" + GDRIVE_DATASET_NAME_TRAIN + ".parquet"
GDRIVE_DATASET_NAME_EXT_VALID = "/" + GDRIVE_DATASET_NAME_VALID + ".parquet"

GDRIVE_DATASET = GDRIVE_DATASET_RAW_DIR + GDRIVE_DATASET_NAME_EXT
GDRIVE_DATASET_TRAIN = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_TRAIN
GDRIVE_DATASET_VALID = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_VALID

In [ ]:
# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


# Training the model ❗

In [ ]:
# Load datasets into pyspark dataframe objects
train_df = spark.read.load(GDRIVE_DATASET_TRAIN,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

valid_df = spark.read.load(GDRIVE_DATASET_VALID,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

In [ ]:
# Function that create a simple rf model (with no hyperparameter tuning)
def simple_random_forest_model(train, featureCol, labelCol):
  rf = RandomForestRegressor(featuresCol=featureCol, labelCol=labelCol)
  rf_model = rf.fit(train)
  return rf_model

In [ ]:
# Define the evaluation metrics
# Notice that r2_adj metric is included when calculating r2
metrics = ['mse', 'rmse', 'mae', 'r2']

In [ ]:
# Function to compute the r2 adjusted metric
# r2 is the r2 metric, n is the number of observations, k is the number of features
def compute_r2adj(r2, n, k):
  return 1 - (1 - r2) * ((n - 1) / (n - k - 1))

In [ ]:
# Function to evaluate a model
def evaluate_models(predictions, modelName, typeName, label, prediction, metrics):
  r2 = None
  for metric in metrics:
    evaluator = RegressionEvaluator(labelCol=label, predictionCol=prediction, metricName=metric)
    evaluation = evaluator.evaluate(predictions)
    print(metric.upper()+' for '+modelName+' on '+typeName+' set: '+str(evaluation))
    if metric == 'r2':
      print('R2_adj'+' for '+modelName+' on '+typeName+' set: '+str(compute_r2adj(evaluation, predictions.count(), len(predictions.columns))))

In [ ]:
# Function that create simple models (without hyperparameter tuning) and evaluate them
def test_best_features(train_data, valid_data, features, featureCol, labelCol, metrics = ['rmse', 'r2']):
  # Train the models
  rf = simple_random_forest_model(train_data, featureCol, labelCol)

  # Training set evaluation
  rf_training = rf.transform(train_data)
  evaluate_models(rf_training, 'random forest regression', 'training', labelCol, 'prediction', metrics)

  # validation set evaluation
  rf_predictions = rf.transform(valid_data)
  evaluate_models(rf_predictions, 'random forest regression', 'validation', labelCol, 'prediction', metrics)

  return rf_training, rf_predictions

In [ ]:
# Retrieve all / cor_matrix / gb features
GDRIVE_FEATURES_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/features"

GDRIVE_COR_MATRIX_FEATURES_NAME = "cor_matrix_features"
GDRIVE_GB_FEATURES_NAME = "gb_features"

GDRIVE_COR_MATRIX_FEATURES_NAME_EXT = "/" + GDRIVE_COR_MATRIX_FEATURES_NAME + ".json"
GDRIVE_GB_FEATURES_NAME_EXT = "/" + GDRIVE_GB_FEATURES_NAME + ".json"

GDRIVE_COR_MATRIX_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_COR_MATRIX_FEATURES_NAME_EXT
GDRIVE_GB_FEATURES = GDRIVE_FEATURES_DIR + GDRIVE_GB_FEATURES_NAME_EXT

In [ ]:
cor_matrix_features = spark.read.json(GDRIVE_COR_MATRIX_FEATURES).columns
gb_features = spark.read.json(GDRIVE_GB_FEATURES).columns

all_features = train_df.columns[1:-2]

# Set the depended variable
dep_var = 'market-price'

In [ ]:
# Return the dataset with the selected features
def select_features(dataset, features):
  vectorAssembler = VectorAssembler(
    inputCols = features,
    outputCol = 'features')

  dataset = vectorAssembler.transform(dataset)
  dataset = dataset.select(['timestamp','index', 'features', dep_var])
  return dataset

In [ ]:
def show_results(train, valid, training, predictions):
  trace1 = go.Scatter(
      x = train['timestamp'],
      y = train['market-price'].astype(float),
      mode = 'lines',
      name = 'Train'
  )

  trace2 = go.Scatter(
      x = valid['timestamp'],
      y = valid['market-price'].astype(float),
      mode = 'lines',
      name = 'Validation'
  )

  trace3 = go.Scatter(
      x = training['timestamp'],
      y = training['prediction'].astype(float),
      mode = 'lines',
      name = 'Training'
  )

  trace4 = go.Scatter(
      x = predictions['timestamp'],
      y = predictions['prediction'].astype(float),
      mode = 'lines',
      name = 'Prediction'
  )

  layout = dict(
      title='Train, valid and prediction set with Rangeslider',
      xaxis=dict(
          rangeselector=dict(
              buttons=list([
                  #change the count to desired amount of months.
                  dict(count=1,
                      label='1m',
                      step='month',
                      stepmode='backward'),
                  dict(count=6,
                      label='6m',
                      step='month',
                      stepmode='backward'),
                  dict(count=12,
                      label='1y',
                      step='month',
                      stepmode='backward'),
                  dict(count=36,
                      label='3y',
                      step='month',
                      stepmode='backward'),
                  dict(step='all')
              ])
          ),
          rangeslider=dict(
              visible = True
          ),
          type='date'
      )
  )

  data = [trace1, trace2, trace3, trace4]
  fig = dict(data=data, layout=layout)
  iplot(fig, filename = "Train, valid and prediction set with Rangeslider")

In [ ]:
# valid performances with all the features
rf_training, rf_predictions = test_best_features(select_features(train_df, all_features), select_features(valid_df, all_features), all_features, 'features', dep_var)

RMSE for random forest regression on training set: 902.1864726153018
R2 for random forest regression on training set: 0.9885682688117203
R2_adj for random forest regression on training set: 0.9885675646445699
RMSE for random forest regression on validation set: 10077.831642818173
R2 for random forest regression on validation set: 0.4790119498024684
R2_adj for random forest regression on validation set: 0.47884072985695847


In [ ]:
show_results(train_df.toPandas(), valid_df.toPandas(), rf_training.toPandas(), rf_predictions.toPandas())

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# valid features with the corr matrix features
rf_training, rf_predictions = test_best_features(select_features(train_df, cor_matrix_features), select_features(valid_df, cor_matrix_features), cor_matrix_features, 'features', dep_var)

RMSE for random forest regression on training set: 1147.5595142622658
R2 for random forest regression on training set: 0.9815043406661919
R2_adj for random forest regression on training set: 0.9815032013780547
RMSE for random forest regression on validation set: 7868.3961333689285
R2 for random forest regression on validation set: 0.6824106013886362
R2_adj for random forest regression on validation set: 0.6823062273257299


In [ ]:
show_results(train_df.toPandas(), valid_df.toPandas(), rf_training.toPandas(), rf_predictions.toPandas())

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# valid performances with gb features
rf_training, rf_predictions = test_best_features(select_features(train_df, gb_features), select_features(valid_df, gb_features), gb_features, 'features', dep_var)

RMSE for random forest regression on training set: 1207.8250887707725
R2 for random forest regression on training set: 0.9795106833331485
R2_adj for random forest regression on training set: 0.9795094212405139
RMSE for random forest regression on validation set: 9411.51972232878
R2 for random forest regression on validation set: 0.5456264130185886
R2_adj for random forest regression on validation set: 0.5454770855613185


In [ ]:
show_results(train_df.toPandas(), valid_df.toPandas(), rf_training.toPandas(), rf_predictions.toPandas())

Output hidden; open in https://colab.research.google.com to view.

# Hyperparameter tuning ❗

In [ ]:
# # Hyperparameter tuning for the model
# def random_forest_cross_val(dataset, k_fold=5):
#     rf = RandomForestRegressor(featuresCol='features', labelCol=dep_var)
#     pipeline = Pipeline(stages=[rf])

#     # Default (too much memory!!)
#     # param_grid = ParamGridBuilder()\
#     # .addGrid(rf.maxDepth, [8, 9, 10]) \
#     # .addGrid(rf.numTrees, [40, 60, 80]) \
#     # .build()

#     param_grid = ParamGridBuilder()\
#     .addGrid(rf.maxDepth, [5, 6, 7]) \
#     .addGrid(rf.numTrees, [30, 50, 70]) \
#     .build()

#     cross_val = CrossValidator(estimator=pipeline,
#                                estimatorParamMaps=param_grid,
#                                evaluator=RegressionEvaluator(labelCol=dep_var),
#                                numFolds=k_fold,
#                                collectSubModels=True
#                                )

#     # Run cross-validation, and choose the best set of parameters.
#     cv_model = cross_val.fit(dataset)

#     return cv_model

In [ ]:
# # Execute cross validation with random forest
# cv_rf_models = random_forest_cross_val(select_features(train_df, cor_matrix_features))

In [ ]:
# # Summarizes all the models trained during cross validation
# def summarize_rf_models(cv_models):
#     for k, models in enumerate(cv_models):
#         print("*************** Fold #{:d} ***************\n".format(k+1))
#         for i, m in enumerate(models):
#             print("--- Model #{:d} out of {:d} ---".format(i+1, len(models)))
#             print("\tParameters: maxDepth=[{:.3f}]; numTrees=[{:.3f}] ".format(m.stages[-1]._java_obj.getMaxDepth(), m.stages[-1]._java_obj.getNumTrees()))
#             print("\tModel summary: {}\n".format(m.stages[-1]))
#         print("***************************************\n")

In [ ]:
# # Call the function above|
# summarize_rf_models(cv_rf_models.subModels)

In [ ]:
# # Summarize average error
# for i, avg_rmse in enumerate(cv_rf_models.avgMetrics):
#     print("Avg. RMSE computed across k-fold cross validation for model setting #{:d}: {:3f}".format(i+1, avg_rmse))

NameError: ignored

In [ ]:
# # Get the best model to extract best hyperparameters
# best_rf_model_params = cv_rf_models.bestModel.stages[-1].extractParamMap()
# # parameters = [] ❗

# print('Best parameters for random forest regressor:')
# for param, value in best_rf_model_params.items():
#     # parameters.append(param.name + "=" + str(value)) ❗
#     print(param.name, "=", value)

In [ ]:
# parameters = [parameter.replace("'", "") for parameter in parameters] ❗
# parameters ❗

In [ ]:
# Fit a model with best parameters
rf = RandomForestRegressor(featuresCol='features', labelCol=dep_var, maxDepth=7, numTrees=50)
pipeline = Pipeline(stages=[rf])

rf_model = pipeline.fit(select_features(valid_df, cor_matrix_features))

In [ ]:
# Training set evaluation
rf_training = rf_model.transform(select_features(train_df, cor_matrix_features))
evaluate_models(rf_training, 'random forest regressor', 'training', dep_var, 'prediction', metrics)

# Validation set evaluation
rf_predictions = rf_model.transform(select_features(valid_df, cor_matrix_features))
evaluate_models(rf_predictions, 'random forest regressor', 'validation', dep_var, 'prediction', metrics)

MSE for random forest regressor on training set: 671989542.1124626
RMSE for random forest regressor on training set: 25922.761081961595
MAE for random forest regressor on training set: 25285.079473176822
R2 for random forest regressor on training set: -8.438041791134859
R2_adj for random forest regressor on training set: -8.438623151812871
MSE for random forest regressor on validation set: 417993.9434295006
RMSE for random forest regressor on validation set: 646.5245110817536
MAE for random forest regressor on validation set: 451.2732916933389
R2 for random forest regressor on validation set: 0.9978558085823623
R2_adj for random forest regressor on validation set: 0.9978551039052828


In [ ]:
show_results(train_df.toPandas(), valid_df.toPandas(), rf_training.toPandas(), rf_predictions.toPandas())

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
GDRIVE_MODEL_NAME = "random_forest"
GDRIVE_MODELS_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/models"
GDRIVE_MODEL_NAME_EXT = GDRIVE_MODELS_DIR + "/" + GDRIVE_MODEL_NAME

In [ ]:
# Save the RF best model
rf_model.write().overwrite().save(GDRIVE_MODEL_NAME_EXT)